The following lines will download the Kaggle dataset using our API token.

In [ ]:
! pip install kaggle -q
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! kaggle datasets download -d arevel/chess-games
! unzip -qq /content/chess-games.zip

Here are all of the other modules that we'll need for our engine.

In [ ]:
! pip install chess -q
! pip install numpy -q
! pip install pandas -q
! pip install torch -q

In [ ]:
import chess
import numpy as np
import pandas as pd
import torch
import re
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

Step 1: Preprocessing

In [ ]:
letter_to_num = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
}

num_to_letter = {
    0: 'a',
    1: 'b',
    2: 'c',
    3: 'd',
    4: 'e',
    5: 'f',
    6: 'g',
    7: 'h',
}

In [ ]:
def board_to_rep(board):
  pieces = ['p', 'n', 'b', 'r', 'q', 'k']
  layers = []

  for piece in pieces:
    layers.append(create_rep_layer(board, piece))

  board_rep = np. stack(layers)
  return layers

In [ ]:
def create_rep_layer(board, type):
  s = str(board)

  s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
  s = re.sub(f'{type}', '-1', s)
  s = re.sub(f'{type.upper()}', 1, s)
  s = re.sub(f'\.', '0', s)

  matrix = []
  for row in s.split('\n'):
    row = row.split(" ")
    row = [int(x) for x in row]
    matrix.append(row)

  return np.array(matrix)

In [ ]:
def move_to_rep(board, move):
  board.push_san(move).uci()
  move = str(board.pop())

  pre_layer = np.zeros((8, 8))
  pre_row = 8 - int(move[1])
  pre_col = letter_to_num[move[0]]
  pre_layer[pre_row, pre_col] = 1

  post_layer = np.zeros((8, 8))
  post_row = 8 - int(move[3])
  post_col = letter_to_num[move[2]]
  post_layer[post_row, post_col] = 1

  return np.stack([pre_layer, post_layer])

In [ ]:
def move_list(s):
  return re.sub('\d*\. ', '', s).split(' ')[:-1]

In [ ]:
# Grabbing higher-level games from the dataset.

raw_data = pd.read_csv('/content/chess_games.csv', usecols=['AN', 'WhiteElo'])
data = raw_data[raw_data['WhiteElo'] > 1800]
del raw_data
gc.collect()

data = data[['AN']]
data = data[-data['AN'].str.contains('{')]
data = data[data['AN'].str.len() > 20]
print("# Games:", data.shape[0])


Part 2: Using PyTorch

In [ ]:
class CDataset(Dataset):

  def __init__(self, games):
    super(CDataset, self).__init__()
    self.games = games

  def __len__(self):
    return 50_000

  def __getitem__(self, idx):
    game_i = np.random.randint(self.games.shape[0])
    random_game = data['AN'].values[game_i]

    moves = move_list(random_game)
    game_state_i = np.random.randint(len(moves) - 1)
    next = moves[game_state_i]
    moves = moves[:game_state_i]
    board = chess.Board()

    for move in moves:
      board.push_san(move)

    x = board_to_rep(board)
    y = move_to_rep(next, board)

    if game_state_i % 2 == 1:
      x *= -1

    return x, y

In [ ]:
data_train = CDataset(data['AN'])
data_loader = DataLoader(data_train, batch_size = 32, shuffle = True, drop_last = True)

In [ ]:
class module(nn.Module):

  def __init__(self, sz):
    super(module, self).__init__()
    self.conv1 = nn.Conv2d(sz, sz, 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(sz, sz, 3, stride = 1, padding = 1)
    self.bn1 = nn.BatchNorm2d(sz)
    self.bn2 = nn.BatchNorm2d(sz)
    self.activation1 = nn.SELU()
    self.activation2 = nn.SELU()

  def forward(self, x):
    x_inp = torch.clone()
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.activation1(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = x + x_inp
    x = self.activation2(x)
    return x

In [ ]:
class ChessNet(nn.Module):
  def __init__(self, hidden_layers = 4, hidden_size = 200):
    super(ChessNet, self).__init__()
    self.hidden_layers = hidden_layers
    self.input_layer = nn.Conv2d(6, hidden_size, 3, stride = 1, padding = 1)
    self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
    self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride = 1, padding = 1)

  def forward(self, x):
    x = self.input_layer(x)
    x = F.relu(x)

    for i in range(self.hidden_layers):
      x = self.module_list[i](x)

    x = self.output_layer(x)

    return x

Now, this is where the magic happens.

In [ ]:
model = ChessNet()
criterion = nn.CrossEntropyLoss()
print_interval = 100

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if batch_idx % print_interval == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item()}")